In [1]:
import os
import sqlite3
from flask import Flask, jsonify, request, render_template_string
from threading import Thread
from werkzeug.serving import make_server
from IPython.display import display, HTML
from datetime import datetime

app = Flask(__name__)

BASE_DIR = os.getcwd()
DB_FILE = os.path.join(BASE_DIR, "chatbot_manicure.db") # Alterado o nome do arquivo do DB

def init_db():
    # A remoção da linha abaixo garante que o banco de dados e os dados persistam entre as reinicializações.
    # if os.path.exists(DB_FILE):
    #    os.remove(DB_FILE)

    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS chat (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            pergunta TEXT NOT NULL UNIQUE,
            resposta TEXT NOT NULL
        )
    ''')

    c.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            phone TEXT UNIQUE NOT NULL,
            name TEXT,
            street TEXT,
            number TEXT,
            neighborhood TEXT
        )
    ''')

    c.execute('''
        CREATE TABLE IF NOT EXISTS chat_history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_phone TEXT NOT NULL,
            message_text TEXT NOT NULL,
            sender TEXT NOT NULL,
            timestamp TEXT NOT NULL,
            FOREIGN KEY (user_phone) REFERENCES users(phone)
        )
    ''')
    
    # NOVAS PERGUNTAS E RESPOSTAS PARA MANICURE
    exemplos = [
        ("olá", "Olá! Bem-vinda(o) à nossa esmalteria! Como posso ajudar você hoje? 💅"),
        ("serviços", "Oferecemos manicure....R$ 35,00, pedicure....R$ 40,00 , esmaltação em gel....R$ 60,00, blindagem....R$ 80,00. Agende seu horário agora mesmo!"),
        ("agendar horário", "Para agendar seu horário, por favor, clique em 'Falar com a Manicure' para agendamento direto!"),
        ("promoção", "Toda terça-feira, pé e mão por apenas R$ 65,00! Aproveite!"),
        ("localização", "Estamos localizadas na [Seu Endereço Completo], [Sua Cidade] - [Seu Estado]. Você pode nos encontrar facilmente no mapa: <a href='[Link do Google Maps]' target='_blank' style='color: #008080; text-decoration: underline; font-weight: bold;'>Ver no Mapa</a>."),
        ("falar com a manicure", "Para falar diretamente com a equipe, clique no link do WhatsApp. 😊"),
        ("encerrar chat", "Foi um prazer ajudar! Esperamos você em breve! 😊"),
        ("horário de funcionamento", "Nosso horário de funcionamento é de segunda a sábado, das 9h às 19h."),
    ]

    for pergunta, resposta in exemplos:
        # Usando INSERT OR IGNORE para evitar duplicatas se o DB já existir com esses valores
        c.execute("INSERT OR IGNORE INTO chat (pergunta, resposta) VALUES (?, ?)", (pergunta, resposta))
            
    conn.commit()
    conn.close()

init_db()

HTML_TEMPLATE = '''
<!DOCTYPE html>
<html lang="pt-BR">
<head>
<meta charset="UTF-8" />
<meta name="viewport" content="width=device-width, initial-scale=1" />
<title>Sua Esmalteria - Chatbot</title>
<style>
    body { 
        font-family: 'Segoe UI', sans-serif; 
        margin: 0; 
        height: 100vh; /* Garante que o body ocupe 100% da altura da viewport */
        background-color: #F0FFF0; /* Cor de fallback caso a imagem não carregue */
        background-image: url('https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSpCGl4EASzPI25zX-yt98AUy83p1GbA5ebFA&s'); /* URL da sua imagem de fundo */
        background-size: cover; /* Faz a imagem cobrir toda a área */
        background-repeat: no-repeat; /* Evita que a imagem se repita */
        background-position: center center; /* Centraliza a imagem */
        background-attachment: fixed; /* Mantém a imagem fixa ao rolar */
    } 
    header, footer { background-color: #40E0D0; color: white; text-align: center; padding: 20px; } /* Verde água vibrante para cabeçalho e rodapé */
    main { padding: 40px; max-width: 800px; margin: auto; }
    h1 { color: #20B2AA; } /* Verde água um pouco mais escuro para o título principal */
    #chatbot-toggle {
        position: fixed;
        bottom: 25px;
        right: 25px;
        width: 60px; height: 60px;
        border-radius: 50%;
        background: #40E0D0; /* Verde água para o botão de toggle */
        display: flex; justify-content: center; align-items: center;
        cursor: pointer;
        z-index: 999;
        box-shadow: 0 6px 12px rgba(64, 224, 208, 0.4); /* Sombra com base no verde água */
    }
    #chatbot-toggle svg { width: 30px; height: 30px; fill: white; }

    #chatbot-box {
        position: fixed;
        bottom: 100px;
        right: 25px;
        width: 350px;
        max-height: 500px;
        background: white;
        border-radius: 20px;
        box-shadow: 0 10px 25px rgba(0,0,0,0.15);
        display: none;
        flex-direction: column;
        overflow: hidden;
        z-index: 1000;
    }
    #chatbot-header {
        background: #40E0D0; /* Verde água para o cabeçalho do chatbot */
        color: white;
        padding: 15px;
        font-weight: bold;
        display: flex;
        justify-content: space-between;
        border-top-left-radius: 20px;
        border-top-right-radius: 20px;
    }
    #chat-log {
        padding: 10px;
        overflow-y: auto;
        flex-grow: 1;
        background: #F0FFF0; /* Fundo suave do log de chat, verde água bem clarinho */
        display: flex;
        flex-direction: column;
        gap: 10px;
    }
    .message {
        max-width: 80%;
        padding: 12px;
        border-radius: 16px;
        font-size: 15px;
    }
    .user { align-self: flex-end; background: #FFB6C1; color: #3A3A3A; } /* Rosa claro para mensagens do usuário */
    .bot { align-self: flex-start; background: #E0FFFF; color: #3A3A3A; } /* Verde água muito claro para mensagens do bot */
    
    .bot a {
        color: #008080; /* Teal escuro para links do bot */
        text-decoration: underline;
        font-weight: bold;
    }
    .bot a:hover {
        color: #005050; /* Teal ainda mais escuro no hover */
    }

    .menu-message {
        align-self: flex-start;
        background: #E0FFFF; /* Verde água muito claro para o menu */
        border-radius: 16px;
        padding: 12px;
        max-width: 80%;
        font-size: 15px;
        display: flex;
        flex-direction: column;
        gap: 8px;
    }

    .menu-message .menu-title {
        margin-bottom: 5px;
        font-weight: bold;
        color: #008080; /* Teal escuro para o título do menu */
    }

    .menu-message button {
        margin: 0;
        padding: 10px 15px;
        border-radius: 20px;
        border: none;
        background: #40E0D0; /* Verde água para botões do menu */
        color: white;
        cursor: pointer;
        font-size: 15px;
        text-align: center;
        width: 100%;
        box-sizing: border-box;
        box-shadow: 0 4px 8px rgba(64, 224, 208, 0.3); /* Sombra com base no verde água */
    }
    .menu-message button:hover {
        background: #20B2AA; /* Verde água um pouco mais escuro no hover */
    }

    #input-area {
        display: flex;
        padding: 10px;
        border-top: 1px solid #E0E0E0;
        background: #fff;
        border-bottom-left-radius: 20px;
        border-bottom-right-radius: 20px;
    }
    #user-input {
        flex-grow: 1;
        padding: 10px;
        border: 1px solid #B0E0E6; /* Borda sutil em azul-água claro */
        border-radius: 25px;
        font-size: 15px;
    }
    .send-btn {
        margin-left: 10px;
        padding: 10px 18px;
        background: #40E0D0; /* Verde água para o botão de enviar */
        color: white;
        border: none;
        border-radius: 25px;
        cursor: pointer;
        box-shadow: 0 4px 8px rgba(64, 224, 208, 0.3); /* Sombra com base no verde água */
    }
    .send-btn:hover {
        background: #20B2AA; /* Verde água um pouco mais escuro no hover */
    }
</style>
</head>
<body>

<header>Sua Esmalteria - Chatbot</header>
<main>
    <h1>Bem-vinda(o) à Sua Esmalteria!</h1>
    <p>Use o chatbot no canto inferior direito para agendar seus serviços, tirar dúvidas e muito mais!</p>
</main>
<footer>&copy; 2025 Sua Esmalteria </footer>
<div id="chatbot-toggle" onclick="toggleChatbot()">
    <svg width="30" height="30" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg">
        <path d="M12 2C8.686 2 6 4.686 6 8C6 11.314 12 22 12 22C12 22 18 11.314 18 8C18 4.686 15.314 2 12 2ZM12 11C10.343 11 9 9.657 9 8C9 6.343 10.343 5 12 5C13.657 5 15 6.343 15 8C15 9.657 13.657 11 12 11Z" fill="white"/>
        <path d="M12 9.5C11.1716 9.5 10.5 = 8.82843 10.5 8C10.5 7.17157 11.1716 6.5 12 6.5C12.8284 6.5 13.5 7.17157 13.5 8C13.5 8.82843 12.8284 9.5 12 9.5Z" fill="#F0F0F0"/>
    </svg>
</div>

<div id="chatbot-box">
    <div id="chatbot-header">
        Chatbot 🌸
        <button onclick="toggleChatbot()">×</button>
    </div>
    <div id="chat-log"></div>
    <div id="input-area">
        <input type="text" id="user-input" placeholder="Digite sua mensagem..." />
        <button class="send-btn" onclick="sendMessage()">Enviar</button>
    </div>
</div>

<script>
    const chatbotBox = document.getElementById('chatbot-box');
    const chatLog = document.getElementById('chat-log');
    const userInput = document.getElementById('user-input');
    const sendBtn = document.querySelector('.send-btn');

    let hasGreeted = false;
    let collectingInfo = false;
    let currentInfoStep = 0;
    let customerName = '';
    let customerPhone = '';
    let customerStreet = '';
    let customerNumber = '';
    let customerNeighborhood = '';
    let customerAddress = '';

    // NOVAS OPÇÕES DE MENU PARA MANICURE
    const menuOptions = [
        { text: 'Nossos Serviços', value: 'serviços' },
        { text: 'Agendar Horário', value: 'agendar horário' },
        { text: 'Promoções', value: 'promoção' },
        { text: 'Localização', value: 'localização' },
        { text: 'Falar com a Manicure', value: 'falar com a manicure' },
        { text: 'Encerrar Chat', value: 'encerrar chat', isSpecial: true }
    ];

    function appendMessage(text, sender) {
        const div = document.createElement('div');
        div.className = 'message ' + sender;
        if (sender === 'bot') {
            div.innerHTML = text;
        } else {
            div.innerText = text;
        }
        chatLog.appendChild(div);
        chatLog.scrollTop = chatLog.scrollHeight;
    }

    function appendMenuMessage(title, options) {
        const menuDiv = document.createElement('div');
        menuDiv.className = 'menu-message bot';

        const titleSpan = document.createElement('span');
        titleSpan.className = 'menu-title';
        titleSpan.innerText = title;
        menuDiv.appendChild(titleSpan);

        options.forEach(option => {
            const button = document.createElement('button');
            button.innerText = option.text;
            button.onclick = () => handleMenuButtonClick(option.value, option.isSpecial);
            menuDiv.appendChild(button);
        });

        chatLog.appendChild(menuDiv);
        chatLog.scrollTop = chatLog.scrollHeight;
    }

    function setInputActive(active) {
        userInput.disabled = !active;
        sendBtn.disabled = !active;
    }

    async function loadChatHistory(phone) {
        try {
            const response = await fetch('/get_chat_history', {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({ phone: phone })
            });
            const data = await response.json();
            if (data.history && data.history.length > 0) {
                chatLog.innerHTML = '';
                data.history.forEach(msg => {
                    appendMessage(msg.message_text, msg.sender);
                });
                appendMessage('Bem-vinda(o) de volta! O que podemos fazer por você hoje?', 'bot');
            } else {
                appendMessage('Bem-vinda(o)! Parece que é sua primeira vez aqui. Como posso ajudar você hoje?', 'bot');
            }
        } catch (error) {
            console.error('Erro ao carregar histórico de chat:', error);
            appendMessage('Desculpe, não consegui carregar seu histórico de chat. Como posso ajudar você?', 'bot');
        }
    }

    async function saveMessageToHistory(userPhone, messageText, sender) {
        try {
            await fetch('/save_chat_message', {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({
                    phone: userPhone,
                    message_text: messageText,
                    sender: sender
                })
            });
        } catch (error) {
            console.error('Erro ao salvar mensagem no histórico:', error);
        }
    }

    function startCollectingInfo() {
        collectingInfo = true;
        currentInfoStep = 0;
        setInputActive(true);
        appendMessage('Olá! Para começarmos, por favor, digite seu número de telefone (apenas números, com DDD).', 'bot');
        userInput.focus();
    }

    async function processCollectedInfo(msg) {
        if (currentInfoStep === 0) {
            if (!/^\d{10,11}$/.test(msg)) {
                appendMessage('Por favor, digite um número de telefone válido (apenas números, com DDD).', 'bot');
            } else {
                customerPhone = msg;
                setInputActive(false);
                appendMessage('Verificando suas informações...', 'bot');

                try {
                    const response = await fetch('/get_user_data', {
                        method: 'POST',
                        headers: { 'Content-Type': 'application/json' },
                        body: JSON.stringify({ phone: customerPhone })
                    });
                    const userData = await response.json();

                    if (userData.found) {
                        customerName = userData.name;
                        customerStreet = userData.street || '';
                        customerNumber = userData.number || '';
                        customerNeighborhood = userData.neighborhood || '';
                        customerAddress = `Rua: ${customerStreet}, Nº: ${customerNumber}, Bairro: ${customerNeighborhood}`;

                        collectingInfo = false;
                        currentInfoStep = 0;
                        appendMessage(`Bem-vinda(o) de volta, ${customerName}!`, 'bot');
                        await loadChatHistory(customerPhone);
                        appendMenuMessage('O que podemos fazer por você hoje?', menuOptions);
                    } else {
                        appendMessage('Número não encontrado. Vamos fazer seu cadastro!', 'bot');
                        currentInfoStep = 1;
                        appendMessage('Qual é o seu nome completo?', 'bot');
                    }
                } catch (error) {
                    console.error('Erro ao verificar telefone:', error);
                    appendMessage('Desculpe, houve um erro ao verificar seu telefone. Por favor, tente novamente.', 'bot');
                } finally {
                    setInputActive(true);
                    userInput.focus();
                }
            }
        } else if (currentInfoStep === 1) {
            const nameRegex = /^[A-Za-z\u00C0-\u00FF\s]+$/;
            if (!nameRegex.test(msg)) {
                appendMessage('Por favor, digite um nome válido, usando apenas letras e espaços.', 'bot');
            } else if (msg.length < 2) {
                appendMessage('Por favor, digite um nome mais completo.', 'bot');
            } else {
                customerName = msg;
                appendMessage(`Ok, ${customerName}!`, 'bot'); 
                currentInfoStep = 2;
                appendMessage('Por favor, digite o nome da sua rua.', 'bot');
            }
        } else if (currentInfoStep === 2) {
            if (msg.length < 3) {
                appendMessage('Por favor, digite um nome de rua válido e mais completo.', 'bot');
            } else {
                customerStreet = msg;
                appendMessage('Rua registrada.', 'bot');
                currentInfoStep = 3;
                appendMessage('Qual é o número da sua casa ou apartamento? Se for apartamento, inclua o número dele (Ex: 123 ou 123, apto 401).', 'bot');
            }
        } else if (currentInfoStep === 3) {
            const numberRegex = /^(\d+)(\s*,\s*apto\s*\d+)?$/i;
            if (!numberRegex.test(msg)) {
                appendMessage('Por favor, digite um número válido para sua casa ou apartamento (Ex: 123 ou 123, apto 401).', 'bot');
            } else {
                customerNumber = msg;
                appendMessage('Número registrado.', 'bot');
                currentInfoStep = 4;
                appendMessage('Qual é o seu bairro?', 'bot');
            }
        } else if (currentInfoStep === 4) {
            if (msg.length < 3) {
                appendMessage('Por favor, digite um nome de bairro válido e mais completo.', 'bot');
            } else {
                customerNeighborhood = msg;
                customerAddress = `Rua: ${customerStreet}, Nº: ${customerNumber}, Bairro: ${customerNeighborhood}`;
                appendMessage('Bairro registrado.', 'bot');
                
                try {
                    await fetch('/save_user_data', {
                        method: 'POST',
                        headers: { 'Content-Type': 'application/json' },
                        body: JSON.stringify({
                            phone: customerPhone,
                            name: customerName,
                            street: customerStreet,
                            number: customerNumber,
                            neighborhood: customerNeighborhood
                        })
                    });
                    appendMessage('Seu cadastro foi concluído com sucesso!', 'bot');
                } catch (error) {
                    console.error('Erro ao salvar dados do usuário:', error);
                    appendMessage('Desculpe, houve um erro ao salvar seus dados. Por favor, tente novamente.', 'bot');
                }

                collectingInfo = false;
                currentInfoStep = 0;
                appendMessage('Agora, como posso ajudar você?', 'bot');
                appendMenuMessage('Selecione uma opção:', menuOptions);
            }
        }
        userInput.focus();
    }

    function handleMenuButtonClick(value, isSpecial) {
        setInputActive(false);

        if (customerPhone) {
            saveMessageToHistory(customerPhone, menuOptions.find(opt => opt.value === value)?.text || value, 'user');
        }
        appendMessage(menuOptions.find(opt => opt.value === value)?.text || value, 'user');

        // Alterado para 'falar com a manicure'
        if (value === 'falar com a manicure') {
            const whatsappNumber = '5511999999999'; // SEU NÚMERO DE WHATSAPP AQUI
            const whatsappLink = `https://wa.me/${whatsappNumber}?text=Olá,%20gostaria%20de%20falar%20com%20a%20manicure%20da%20Esmalteria.`; // Texto ajustado
            const botResponse = `Certo! Para falar com a equipe, clique no link a seguir: <a href="${whatsappLink}" target="_blank" style="color: #008080; text-decoration: underline; font-weight: bold;">Falar com a Manicure no WhatsApp</a>`; // Texto ajustado
            appendMessage(botResponse, 'bot');
            if (customerPhone) {
                saveMessageToHistory(customerPhone, botResponse, 'bot');
            }
            appendMenuMessage('Selecione outra opção:', menuOptions);
            setInputActive(true);
            userInput.focus();
            return;
        } else if (isSpecial && value === 'encerrar chat') {
            fetch('/chat', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({ pergunta: value })
            })
            .then(res => res.json())
            .then(data => {
                appendMessage(data.resposta, 'bot');
                if (customerPhone) {
                    saveMessageToHistory(customerPhone, data.resposta, 'bot');
                }
                setTimeout(() => {
                    toggleChatbot();
                    chatLog.innerHTML = '';
                    hasGreeted = false;
                    customerName = '';
                    customerPhone = '';
                    customerStreet = '';
                    customerNumber = '';
                    customerNeighborhood = '';
                    customerAddress = '';
                    currentInfoStep = 0;
                    collectingInfo = false;
                    setInputActive(false);
                }, 1500);
            }).catch(error => {
                console.error('Erro ao encerrar chat:', error);
                appendMessage('Desculpe, ocorreu um erro ao encerrar o chat.', 'bot');
                setInputActive(true);
            });
        }
        else {
            fetch('/chat', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({ pergunta: value })
            })
            .then(res => res.json())
            .then(data => {
                appendMessage(data.resposta, 'bot');
                if (customerPhone) {
                    saveMessageToHistory(customerPhone, data.resposta, 'bot');
                }
                appendMenuMessage('Selecione outra opção:', menuOptions);
                setInputActive(true);
                userInput.focus();
            }).catch(error => {
                console.error('Erro ao processar opção do menu:', error);
                appendMessage('Desculpe, ocorreu um erro ao processar sua seleção.', 'bot');
                setInputActive(true);
            });
        }
    }

    function toggleChatbot() {
        console.log('toggleChatbot chamado. Display atual:', chatbotBox.style.display);
        if (chatbotBox.style.display === 'flex') {
            chatbotBox.style.display = 'none';
            console.log('Chatbot oculto.');
        } else {
            chatbotBox.style.display = 'flex';
            console.log('Chatbot visível. hasGreeted:', hasGreeted);
            if (!hasGreeted) {
                startCollectingInfo();
                hasGreeted = true;
            } else {
                setInputActive(true);
                if (!collectingInfo && currentInfoStep === 0) { 
                    appendMessage('Olá novamente! Como posso ajudar você?', 'bot');
                    appendMenuMessage('Selecione uma opção:', menuOptions);
                } else if (collectingInfo) {
                    if (currentInfoStep === 0) {
                        appendMessage('Ainda estou esperando seu número de telefone, por favor.', 'bot');
                    } else if (currentInfoStep === 1) {
                        appendMessage('Ainda estou esperando seu nome completo, por favor.', 'bot');
                    } else if (currentInfoStep === 2) {
                        appendMessage('Ainda estou esperando o nome da sua rua, por favor.', 'bot');
                    } else if (currentInfoStep === 3) {
                        appendMessage('Ainda estou esperando o número da sua casa ou apartamento, por favor.', 'bot');
                    } else if (currentInfoStep === 4) {
                        appendMessage('Ainda estou esperando o nome do seu bairro, por favor.', 'bot');
                    }
                }
                userInput.focus();
            }
        }
    }

    async function sendMessage() {
        const msg = userInput.value.trim();
        if (!msg) return;

        if (customerPhone) {
            await saveMessageToHistory(customerPhone, msg, 'user');
        }
        
        appendMessage(msg, 'user');
        userInput.value = '';

        if (collectingInfo) {
            await processCollectedInfo(msg);
            return;
        }
        
        let payload = { pergunta: msg }; 

        setInputActive(false);

        fetch('/chat', {
            method: 'POST',
            headers: {'Content-Type': 'application/json'},
            body: JSON.stringify(payload)
        })
        .then(res => {
            if (!res.ok) {
                throw new Error(`Erro de rede ou servidor: ${res.status}`);
            }
            return res.json();
        })
        .then(async data => {
            appendMessage(data.resposta, 'bot');
            if (customerPhone) {
                await saveMessageToHistory(customerPhone, data.resposta, 'bot');
            }
            
            if (payload.pergunta !== 'encerrar chat') {
                appendMenuMessage('Selecione outra opção:', menuOptions);
            }
            setInputActive(true);
            userInput.focus();
        })
        .catch(error => {
            console.error('Erro ao comunicar com o servidor:', error);
            appendMessage('Desculpe, ocorreu um erro ao processar sua solicitação. Por favor, tente novamente.', 'bot');
            setInputActive(true);
            userInput.focus();
        });
    }

    userInput.addEventListener('keydown', function(event) {
        if (event.key === 'Enter') {
            sendMessage();
            event.preventDefault();
        }
    });

    window.onload = function() {
        setInputActive(false);
    };
</script>

</body>
</html>
'''

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/get_user_data', methods=['POST'])
def get_user_data():
    data = request.json
    phone = data.get('phone', '')
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.execute("SELECT name, street, number, neighborhood FROM users WHERE phone=?", (phone,))
    user = c.fetchone()
    conn.close()
    if user:
        return jsonify({
            'found': True,
            'name': user[0],
            'street': user[1],
            'number': user[2],
            'neighborhood': user[3]
        })
    return jsonify({'found': False})

@app.route('/save_user_data', methods=['POST'])
def save_user_data():
    data = request.json
    phone = data.get('phone', '')
    name = data.get('name', '')
    street = data.get('street', '')
    number = data.get('number', '')
    neighborhood = data.get('neighborhood', '')

    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    try:
        c.execute("INSERT INTO users (phone, name, street, number, neighborhood) VALUES (?, ?, ?, ?, ?)",
                    (phone, name, street, number, neighborhood))
        conn.commit()
        return jsonify({'success': True})
    except sqlite3.IntegrityError:
        return jsonify({'success': False, 'message': 'Telefone já cadastrado.'})
    finally:
        conn.close()

@app.route('/save_chat_message', methods=['POST'])
def save_chat_message():
    data = request.json
    user_phone = data.get('phone', '')
    message_text = data.get('message_text', '')
    sender = data.get('sender', '')
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    try:
        c.execute("INSERT INTO chat_history (user_phone, message_text, sender, timestamp) VALUES (?, ?, ?, ?)",
                    (user_phone, message_text, sender, timestamp))
        conn.commit()
        return jsonify({'success': True})
    except Exception as e:
        print(f"Erro ao salvar mensagem: {e}")
        return jsonify({'success': False, 'message': str(e)})
    finally:
        conn.close()

@app.route('/get_chat_history', methods=['POST'])
def get_chat_history():
    data = request.json
    phone = data.get('phone', '')
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.execute("SELECT message_text, sender FROM chat_history WHERE user_phone=? ORDER BY timestamp", (phone,))
    history = [{'message_text': row[0], 'sender': row[1]} for row in c.fetchall()]
    conn.close()
    return jsonify({'history': history})

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    pergunta = data.get("pergunta", "").lower().strip()

    resposta = "Desculpe, não entendi sua pergunta. 🤔"
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()

    c.execute("SELECT resposta FROM chat WHERE LOWER(pergunta)=?", (pergunta,))
    row = c.fetchone()
    if row:
        resposta = row[0]

    conn.close()
    return jsonify({"resposta": resposta})

class ServerThread(Thread):
    def __init__(self, app):
        Thread.__init__(self)
        self.srv = make_server('0.0.0.0', 5002, app)
        self.ctx = app.app_context()
        self.ctx.push()

    def run(self):
        self.srv.serve_forever()

    def shutdown(self):
        self.srv.shutdown()

server = ServerThread(app)
server.start()

display(HTML("<a href='http://localhost:5002' target='_blank'>🔗 Clique aqui para abrir o site com o chatbot da Sua Esmalteria</a>"))


<>:364: SyntaxWarning: invalid escape sequence '\d'
<>:364: SyntaxWarning: invalid escape sequence '\d'
C:\Users\sammy\AppData\Local\Temp\ipykernel_25720\692728664.py:364: SyntaxWarning: invalid escape sequence '\d'
  if (!/^\d{10,11}$/.test(msg)) {
